In [1]:
import pandas as pd
import os

In [2]:
DIR = 'C:/Users/USER/Desktop/python/competition/CarCrashAnalysisAIContest/data/'
os.chdir(DIR)
os.getcwd()
df = pd.read_csv(DIR+'train.csv')
df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [3]:
df['video_path'] = df['video_path'].apply(lambda x :  './'+'train_thumbnail'+'/'+x.split('/')[2].split('.')[0]+'.jpg')
df

,sample_id,video_path,label
0,TRAIN_0000,./train_thumbnail/TRAIN_0000.jpg,7
1,TRAIN_0001,./train_thumbnail/TRAIN_0001.jpg,7
2,TRAIN_0002,./train_thumbnail/TRAIN_0002.jpg,0
3,TRAIN_0003,./train_thumbnail/TRAIN_0003.jpg,0
4,TRAIN_0004,./train_thumbnail/TRAIN_0004.jpg,1
...,...,...,...
2693,TRAIN_2693,./train_thumbnail/TRAIN_2693.jpg,3
2694,TRAIN_2694,./train_thumbnail/TRAIN_2694.jpg,5
2695,TRAIN_2695,./train_thumbnail/TRAIN_2695.jpg,0
2696,TRAIN_2696,./train_thumbnail/TRAIN_2696.jpg,0


In [4]:
df_timing = df.copy()
def change(x):
    if x == 0:
        return 0
    elif x % 2 == 1:
        return 1
    else:
        return 2

df_timing['label'] = df_timing['label'].apply(change)
def change(x):
    if x == 0:
        return 'Nothing'
    elif x == 1:
        return 'Day'
    else:
        return 'Night'
df_timing['label_name'] = df_timing['label'].apply(change)
df_timing[['label','label_name']].value_counts()

label  label_name
0      Nothing       1783
1      Day            808
2      Night          107
dtype: int64

In [5]:
label2id,id2label = dict(),dict()
for i in df_timing[['label','label_name']].value_counts().items():
    label2id[i[0][1]] = i[0][0]
    id2label[i[0][0]] = i[0][1]
id2label

{0: 'Nothing', 1: 'Day', 2: 'Night'}

In [6]:
df_timing

,sample_id,video_path,label,label_name
0,TRAIN_0000,./train_thumbnail/TRAIN_0000.jpg,1,Day
1,TRAIN_0001,./train_thumbnail/TRAIN_0001.jpg,1,Day
2,TRAIN_0002,./train_thumbnail/TRAIN_0002.jpg,0,Nothing
3,TRAIN_0003,./train_thumbnail/TRAIN_0003.jpg,0,Nothing
4,TRAIN_0004,./train_thumbnail/TRAIN_0004.jpg,1,Day
...,...,...,...,...
2693,TRAIN_2693,./train_thumbnail/TRAIN_2693.jpg,1,Day
2694,TRAIN_2694,./train_thumbnail/TRAIN_2694.jpg,1,Day
2695,TRAIN_2695,./train_thumbnail/TRAIN_2695.jpg,0,Nothing
2696,TRAIN_2696,./train_thumbnail/TRAIN_2696.jpg,0,Nothing


In [7]:
type(list(df_timing['label'].values))

list

In [125]:
df_timing['label'].value_counts()

0    1783
1     808
2     107
Name: label, dtype: int64

In [8]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir="./train_thumbnail", split="train")
dataset

Resolving data files: 100%|██████████| 2698/2698 [00:00<00:00, 23645.19it/s]
Using custom data configuration default-bb5de6f91bbeaf52
Found cached dataset imagefolder (C:/Users/USER/.cache/huggingface/datasets/imagefolder/default-bb5de6f91bbeaf52/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


Dataset({
    features: ['image'],
    num_rows: 2698
})

In [9]:
dataset = dataset.add_column(name='labels',column=list(df_timing['label'].values))
# dataset = dataset.add_column(name='image_file_path',column=list(df_timing['labels'].values))

In [10]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [11]:


def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['labels']
    return inputs

In [12]:
prepared_ds = dataset.with_transform(transform)
prepared_ds = prepared_ds.train_test_split(test_size=0.2)
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 540
    })
})

In [13]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [14]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\USER\AppData\Local\Temp/ipykernel_67456/2425197347.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [15]:
from transformers import ViTForImageClassification

# labels = dataset.features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=3,
    id2label=id2label,
    label2id=label2id)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

Using cuda_amp half precision backend


In [18]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2158
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 540
  Number of trainable parameters = 85800963
  2%|▏         | 10/540 [00:40<32:07,  3.64s/it] 

{'loss': 1.0914, 'learning_rate': 0.0002, 'epoch': 0.07}


  4%|▎         | 20/540 [01:16<31:01,  3.58s/it]

{'loss': 1.0816, 'learning_rate': 0.0002, 'epoch': 0.15}


  6%|▌         | 30/540 [01:52<30:34,  3.60s/it]

...

{'loss': 0.4374, 'learning_rate': 1.074074074074074e-05, 'epoch': 4.0}


100%|██████████| 540/540 [37:44<00:00,  4.19s/it]
Saving model checkpoint to ./
Configuration saved in ./config.json


{'train_runtime': 2264.8912, 'train_samples_per_second': 3.811, 'train_steps_per_second': 0.238, 'train_loss': 0.5854113380114238, 'epoch': 4.0}


Model weights saved in ./pytorch_model.bin
Image processor saved in ./preprocessor_config.json


***** train metrics *****
  epoch                    =        4.0
  train_loss               =     0.5854
  train_runtime            = 0:37:44.89
  train_samples_per_second =      3.811
  train_steps_per_second   =      0.238


In [62]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained('./')

Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin


In [20]:
df_test = pd.read_csv(DIR+'test.csv')
df_test

,sample_id,video_path
0,TEST_0000,./test/TEST_0000.mp4
1,TEST_0001,./test/TEST_0001.mp4
2,TEST_0002,./test/TEST_0002.mp4
3,TEST_0003,./test/TEST_0003.mp4
4,TEST_0004,./test/TEST_0004.mp4
...,...,...
1795,TEST_1795,./test/TEST_1795.mp4
1796,TEST_1796,./test/TEST_1796.mp4
1797,TEST_1797,./test/TEST_1797.mp4
1798,TEST_1798,./test/TEST_1798.mp4


In [98]:
df_test['video_path'] = df_test['video_path'].apply(lambda x :  './'+'test_thumbnail'+'/'+x.split('/')[2].split('.')[0]+'.jpg')
df_test

,sample_id,video_path
0,TEST_0000,./test_thumbnail/TEST_0000.jpg
1,TEST_0001,./test_thumbnail/TEST_0001.jpg
2,TEST_0002,./test_thumbnail/TEST_0002.jpg
3,TEST_0003,./test_thumbnail/TEST_0003.jpg
4,TEST_0004,./test_thumbnail/TEST_0004.jpg
...,...,...
1795,TEST_1795,./test_thumbnail/TEST_1795.jpg
1796,TEST_1796,./test_thumbnail/TEST_1796.jpg
1797,TEST_1797,./test_thumbnail/TEST_1797.jpg
1798,TEST_1798,./test_thumbnail/TEST_1798.jpg


In [80]:
from datasets import load_dataset
dataset_test = load_dataset("imagefolder", data_dir="./test_thumbnail", split="train")
dataset_test

Resolving data files: 100%|██████████| 1777/1777 [00:00<00:00, 30091.40it/s]
Using custom data configuration default-9a237f147f97ad6e
Found cached dataset imagefolder (C:/Users/USER/.cache/huggingface/datasets/imagefolder/default-9a237f147f97ad6e/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


Dataset({
    features: ['image'],
    num_rows: 1777
})

In [81]:
def transform_test(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    # inputs['labels'] = example_batch['labels']
    return inputs

In [85]:
# dataset_test = dataset_test.add_column(name='labels',column=list(df_timing['label'].values))
dataset_test = dataset_test.with_transform(transform_test)
dataset_test

Dataset({
    features: ['image'],
    num_rows: 1777
})

In [90]:
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 540
    })
})

In [91]:
metrics = trainer.evaluate(prepared_ds['test'])

***** Running Evaluation *****
  Num examples = 540
  Batch size = 8
100%|██████████| 68/68 [00:49<00:00,  1.37it/s]


dict_items([('eval_loss', 0.377436101436615), ('eval_accuracy', 0.9388888888888889), ('eval_runtime', 50.7342), ('eval_samples_per_second', 10.644), ('eval_steps_per_second', 1.34)])

In [86]:
def collate_test_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        # 'labels': torch.tensor([x['labels'] for x in batch])
    }

In [87]:
testing_args = TrainingArguments(
  output_dir="./",
  do_train=False,
  do_predict=True,
  per_device_train_batch_size=16,
  # evaluation_strategy="steps",
  # num_train_epochs=4,
  # fp16=True,
  # save_steps=100,
  # eval_steps=100,
  # logging_steps=10,
  # learning_rate=2e-4,
  # save_total_limit=2,
  # remove_unused_columns=False,
  # push_to_hub=False,
  # report_to='tensorboard',
  # load_best_model_at_end=True,
  dataloader_drop_last = False
)
test_trainer = Trainer(
              model = model, 
              args = testing_args, 
              compute_metrics = compute_metrics,
              data_collator=collate_test_fn,
              tokenizer=feature_extractor)
test_result = test_trainer.predict(dataset_test)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `ViTForImageClassification.forward` and have been ignored: image. If image are not expected by `ViTForImageClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 0
  Batch size = 8


In [88]:
len(dataset_test)

1777

In [118]:
def transform_test(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    # inputs['labels'] = example_batch['labels']
    return inputs

In [119]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
model = ViTForImageClassification.from_pretrained('12')

loading configuration file preprocessor_config.json from cache at C:\Users\USER/.cache\huggingface\hub\models--google--vit-base-patch16-224-in21k\snapshots\1ba429d32753f33a0660b80ac6f43a3c80c18938\preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file 12\config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "

In [120]:
from PIL import Image
from tqdm import tqdm
predict = list()
for i in tqdm(df_test.values):
    try:
        img = Image.open(i[1])
        img = img
        inputs = feature_extractor(img, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs).logits
        predict_label = outputs.argmax(-1).item()
        predict.append(predict_label)
    except:
        predict.append(0)


100%|██████████| 1800/1800 [12:56<00:00,  2.32it/s]


In [122]:
len(predict)

1800

In [123]:
set(predict)

{0, 1}